# Notebook
https://www.kaggle.com/williamroe/bi-lstm-with-crf-for-ner

In [3]:
what_corpus = 'polnear'
what_type_files = 'test'

In [4]:
filepath = '../' + what_corpus + '-conll/' + what_type_files + '-conll-foreval/'

In [5]:
import numpy as np
import pandas as pd 
from os import listdir
from os.path import isfile, join

files_list = listdir(filepath)
# files_list = ['wsj_0003.xml.conll.features.foreval']

df_train = pd.DataFrame()
for filename in files_list:  # [0:20]:
    df = pd.read_csv(filepath + filename, sep="\t", header=None, engine='python', names=['col' + str(x) for x in range(1, 30)])
    # Delete all columns fully filled with NaN
    df = df.dropna(axis=1, how='all')
    df_train = df_train.append(df)
    
print(len(df_train.index))
# print(sorted(files_list))

73370


In [22]:
len(files_list)

84

In [23]:
# Convert integer '0's to string, for PARC 3.0 data
df_train['col11'] = df_train['col11'].apply(str)

In [24]:
# Remove two filenames from the label column, for PolNeAR data
df_train["col11"] = df_train["col11"].apply(lambda x: x.replace("breitbart_2016-04-19_clinton-wins-democratic-presiden.txt.xml", "_"))
df_train["col11"] = df_train["col11"].apply(lambda x: x.replace("breitbart_2016-06-29_donald-trump-denies-mike-tyson-i.txt.xml", "_"))

In [25]:
# Remove nested attributes, for PARC 3.0 data
def remove_nested_attributions(value_list):
    return_value = ''
    for value in value_list:
        attr_elements_list = value.split('-')
        if len(attr_elements_list) > 1:
            if attr_elements_list[2] != 'NE':
                return_value = value.replace('-empty','').replace('-AT','').replace('-PD','').replace('-pd','')
    return return_value

df_train["col14"] = df_train["col11"]
df_train["col14"] = df_train.col14.str.split(' ')
df_train["col14"] = df_train['col14'].apply(remove_nested_attributions)
# df_train["col14"] = df_train.col14.str.join(' ')
df_train.tail(20)


,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col14
1910,wash-post_2016-10-27_top-aide-s-leaked-memo-de...,66,1911,37,"10380,10387",company,company,NN,nsubj,38,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ...,I-SOURCE-66
1911,wash-post_2016-10-27_top-aide-s-leaked-memo-de...,66,1912,38,"10388,10394",denied,deny,VBD,acl:relcl,35,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ...,B-CUE-66
1912,wash-post_2016-10-27_top-aide-s-leaked-memo-de...,66,1913,39,"10394,10395",.,.,.,punct,4,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ...,
1913,wash-post_2016-10-27_top-aide-s-leaked-memo-de...,67,1914,1,"10397,10398",``,``,``,punct,16,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ...,B-CONTENT-67
1914,wash-post_2016-10-27_top-aide-s-leaked-memo-de...,67,1915,2,"10398,10400",It,it,PRP,nsubj,3,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ...,I-CONTENT-67
1915,wash-post_2016-10-27_top-aide-s-leaked-memo-de...,67,1916,3,"10401,10408",appears,appear,VBZ,ccomp,16,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ...,I-CONTENT-67
1916,wash-post_2016-10-27_top-aide-s-leaked-memo-de...,67,1917,4,"10409,10412",Dow,Dow,NNP,nsubj,6,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ...,I-CONTENT-67
1917,wash-post_2016-10-27_top-aide-s-leaked-memo-de...,67,1918,5,"10413,10415",is,be,VBZ,aux,6,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ...,I-CONTENT-67
1918,wash-post_2016-10-27_top-aide-s-leaked-memo-de...,67,1919,6,"10416,10422",paying,pay,VBG,ccomp,3,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ...,I-CONTENT-67
1919,wash-post_2016-10-27_top-aide-s-leaked-memo-de...,67,1920,7,"10423,10428",Teneo,Teneo,NNP,dobj,6,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ...,I-CONTENT-67


In [26]:
df_train["col12"] = df_train["col14"].apply(lambda x: x.replace(" ", ""))
df_train["col12"] = df_train["col12"].apply(lambda x: x.replace("_", ""))
df_train["col12"] = df_train["col12"].apply(lambda x: x.replace("B-", ""))
df_train["col12"] = df_train["col12"].apply(lambda x: x.replace("I-", ""))
df_train["col12"] = df_train["col12"].str.replace('\d+', '')
df_train["col12"] = df_train["col12"].apply(lambda x: x.rstrip("-"))
df_train.head(2)

<ipython-input-26-097ab1a9ff7d>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_train["col12"] = df_train["col12"].str.replace('\d+', '')


,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col14,col12
0,huff-post_2016-11-05_hillary-clinton-drops-int...,1,1,1,"0,7",Hillary,Hillary,NNP,compound,5,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _,,
1,huff-post_2016-11-05_hillary-clinton-drops-int...,1,2,2,"8,15",Clinton,Clinton,NNP,compound,5,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _,,


In [27]:
df_train["col13"] = df_train["col14"].apply(lambda x: x.replace(" ", ""))
df_train["col13"] = df_train["col13"].apply(lambda x: x.replace("_", ""))
df_train["col13"] = df_train["col13"].str.replace('\d+', '')
df_train["col13"] = df_train["col13"].apply(lambda x: x.rstrip("-"))
df_train.head(2)

<ipython-input-27-48280b22938a>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_train["col13"] = df_train["col13"].str.replace('\d+', '')


,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col14,col12,col13
0,huff-post_2016-11-05_hillary-clinton-drops-int...,1,1,1,"0,7",Hillary,Hillary,NNP,compound,5,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _,,,
1,huff-post_2016-11-05_hillary-clinton-drops-int...,1,2,2,"8,15",Clinton,Clinton,NNP,compound,5,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _,,,


In [28]:
dataset = df_train[['col1', 'col2', 'col7', 'col12']]
dataset = dataset.rename(columns={'col1': 'filename','col2': 'sentence_idx', 'col7': 'word', 'col12': 'tag'})

In [29]:
dataset[dataset['tag'].isna()]

,filename,sentence_idx,word,tag


In [30]:
from math import nan

tags = []
for index, tag in enumerate(set(dataset["tag"].values)):
    if tag is nan or isinstance(tag, float):
        tags.append('unk')
    else:
        tags.append(tag)
print(tags)

['', 'CUE', 'SOURCE', 'CONTENT']


In [31]:
dataset.head(5)

,filename,sentence_idx,word,tag
0,huff-post_2016-11-05_hillary-clinton-drops-int...,1,Hillary,
1,huff-post_2016-11-05_hillary-clinton-drops-int...,1,Clinton,
2,huff-post_2016-11-05_hillary-clinton-drops-int...,1,drops,
3,huff-post_2016-11-05_hillary-clinton-drops-int...,1,Into,
4,huff-post_2016-11-05_hillary-clinton-drops-int...,1,Detroit,


In [32]:
dataset[dataset['tag'].isna()]

,filename,sentence_idx,word,tag


In [33]:
# from math import nan

classes_map = {
    # Incorrect classes
    "": "O",
    "o": "O",
    "O\n": "O",
    "O": "O",
    None: "O",
    np.NaN: "O",
#     nan: "O",
    # Correct classes
    "CONTENT": "CONTENT",
    "O": "O",
    "SOURCE": "SOURCE",
    "CUE": "CUE",
    # Bio classes
    "B-CONTENT": "B-CONTENT",
    "B-SOURCE": "B-SOURCE",
    "B_CUE": "B-CUE",
    "I-CONTENT": "I-CONTENT",
    "I-SOURCE": "I-SOURCE",
    "I-CUE": "I-CUE",
}

dataset['tag'] = dataset['tag'].map(classes_map)
dataset.head(10)

,filename,sentence_idx,word,tag
0,huff-post_2016-11-05_hillary-clinton-drops-int...,1,Hillary,O
1,huff-post_2016-11-05_hillary-clinton-drops-int...,1,Clinton,O
2,huff-post_2016-11-05_hillary-clinton-drops-int...,1,drops,O
3,huff-post_2016-11-05_hillary-clinton-drops-int...,1,Into,O
4,huff-post_2016-11-05_hillary-clinton-drops-int...,1,Detroit,O
5,huff-post_2016-11-05_hillary-clinton-drops-int...,1,as,O
6,huff-post_2016-11-05_hillary-clinton-drops-int...,1,Democrats,SOURCE
7,huff-post_2016-11-05_hillary-clinton-drops-int...,1,get,CUE
8,huff-post_2016-11-05_hillary-clinton-drops-int...,1,nervous,CUE
9,huff-post_2016-11-05_hillary-clinton-drops-int...,1,about,CUE


In [34]:
dataset[50:70]

,filename,sentence_idx,word,tag
50,huff-post_2016-11-05_hillary-clinton-drops-int...,3,happy,CONTENT
51,huff-post_2016-11-05_hillary-clinton-drops-int...,3,to,CONTENT
52,huff-post_2016-11-05_hillary-clinton-drops-int...,3,be,CONTENT
53,huff-post_2016-11-05_hillary-clinton-drops-int...,3,back,CONTENT
54,huff-post_2016-11-05_hillary-clinton-drops-int...,3,in,CONTENT
55,huff-post_2016-11-05_hillary-clinton-drops-int...,3,Michigan,CONTENT
56,huff-post_2016-11-05_hillary-clinton-drops-int...,3,",",CONTENT
57,huff-post_2016-11-05_hillary-clinton-drops-int...,3,'',CONTENT
58,huff-post_2016-11-05_hillary-clinton-drops-int...,3,Clinton,SOURCE
59,huff-post_2016-11-05_hillary-clinton-drops-int...,3,say,CUE


In [35]:
dataset[dataset['tag'].isna()]

,filename,sentence_idx,word,tag


In [6]:
out_filepath = '../' + what_corpus + '-conll/prepared/'
out_file_name = what_corpus + '_preprocessed_' + what_type_files + '_noBIO.csv'

dataset.to_csv(out_filepath + out_file_name, sep = ';')

NameError: name 'dataset' is not defined

# Merge datasets

In [14]:
import pandas as pd

what_corpus = 'polnear'
filepath = '../' + what_corpus + '-conll/prepared/'

filename = what_corpus + '_preprocessed_train_noBIO.csv'
df_polnear_train = pd.read_csv(filepath + filename, sep=";")

filename = what_corpus + '_preprocessed_test_noBIO.csv'
df_polnear_test = pd.read_csv(filepath + filename, sep=";")

filename = what_corpus + '_preprocessed_dev_noBIO.csv'
df_polnear_val = pd.read_csv(filepath + filename, sep=";")

In [17]:
what_corpus = 'parc30'
filepath = '../' + what_corpus + '-conll/prepared/'

filename = what_corpus + '_preprocessed_train_noBIO.csv'
df_parc30_train = pd.read_csv(filepath + filename, sep=";")

filename = what_corpus + '_preprocessed_test_noBIO.csv'
df_parc30_test = pd.read_csv(filepath + filename, sep=";")

filename = what_corpus + '_preprocessed_dev_noBIO.csv'
df_parc30_val = pd.read_csv(filepath + filename, sep=";")

In [18]:
out_filepath = '../data_total/prepared/'

df_concat = df_polnear_train.append(df_parc30_train)
out_file_name = 'data_merged_preprocessed_train_noBIO.csv'
df_concat.to_csv(out_filepath + out_file_name, sep = ';')

df_concat = df_polnear_test.append(df_parc30_test)
out_file_name = 'data_merged_preprocessed_test_noBIO.csv'
df_concat.to_csv(out_filepath + out_file_name, sep = ';')

df_concat = df_polnear_val.append(df_parc30_val)
out_file_name = 'data_merged_preprocessed_dev_noBIO.csv'
df_concat.to_csv(out_filepath + out_file_name, sep = ';')

In [21]:
print(len(df_concat))
df_concat.head(20)

111200


,Unnamed: 0,filename,sentence_idx,word,tag
0,0,west-journal_2016-09-29_gold-star-mom-corners-...,1,Gold,O
1,1,west-journal_2016-09-29_gold-star-mom-corners-...,1,Star,O
2,2,west-journal_2016-09-29_gold-star-mom-corners-...,1,mom,O
3,3,west-journal_2016-09-29_gold-star-mom-corners-...,1,Corners,O
4,4,west-journal_2016-09-29_gold-star-mom-corners-...,1,Obama,O
5,5,west-journal_2016-09-29_gold-star-mom-corners-...,1,on,O
6,6,west-journal_2016-09-29_gold-star-mom-corners-...,1,he,SOURCE
7,7,west-journal_2016-09-29_gold-star-mom-corners-...,1,refusal,CUE
8,8,west-journal_2016-09-29_gold-star-mom-corners-...,1,to,CONTENT
9,9,west-journal_2016-09-29_gold-star-mom-corners-...,1,use,CONTENT


In [22]:
print(len(df_polnear_val))
df_polnear_val.head(20)

78348


,Unnamed: 0,filename,sentence_idx,word,tag
0,0,west-journal_2016-09-29_gold-star-mom-corners-...,1,Gold,O
1,1,west-journal_2016-09-29_gold-star-mom-corners-...,1,Star,O
2,2,west-journal_2016-09-29_gold-star-mom-corners-...,1,mom,O
3,3,west-journal_2016-09-29_gold-star-mom-corners-...,1,Corners,O
4,4,west-journal_2016-09-29_gold-star-mom-corners-...,1,Obama,O
5,5,west-journal_2016-09-29_gold-star-mom-corners-...,1,on,O
6,6,west-journal_2016-09-29_gold-star-mom-corners-...,1,he,SOURCE
7,7,west-journal_2016-09-29_gold-star-mom-corners-...,1,refusal,CUE
8,8,west-journal_2016-09-29_gold-star-mom-corners-...,1,to,CONTENT
9,9,west-journal_2016-09-29_gold-star-mom-corners-...,1,use,CONTENT


In [23]:
df_concat.tail(20)

,Unnamed: 0,filename,sentence_idx,word,tag
32832,605,wsj_2403.xml,20,advance,O
32833,606,wsj_2403.xml,20,",",O
32834,607,wsj_2403.xml,20,B.A.T,SOURCE
32835,608,wsj_2403.xml,20,have,CUE
32836,609,wsj_2403.xml,20,propose,CUE
32837,610,wsj_2403.xml,20,a,CONTENT
32838,611,wsj_2403.xml,20,sweeping,CONTENT
32839,612,wsj_2403.xml,20,restructuring,CONTENT
32840,613,wsj_2403.xml,20,that,CONTENT
32841,614,wsj_2403.xml,20,would,CONTENT


In [24]:
print(len(df_parc30_val))
df_parc30_val.tail(20)

32852


,Unnamed: 0,filename,sentence_idx,word,tag
32832,605,wsj_2403.xml,20,advance,O
32833,606,wsj_2403.xml,20,",",O
32834,607,wsj_2403.xml,20,B.A.T,SOURCE
32835,608,wsj_2403.xml,20,have,CUE
32836,609,wsj_2403.xml,20,propose,CUE
32837,610,wsj_2403.xml,20,a,CONTENT
32838,611,wsj_2403.xml,20,sweeping,CONTENT
32839,612,wsj_2403.xml,20,restructuring,CONTENT
32840,613,wsj_2403.xml,20,that,CONTENT
32841,614,wsj_2403.xml,20,would,CONTENT
